<a href="https://colab.research.google.com/github/codedfortamara/datasciencecoursera/blob/main/tuning_smartphone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
# IMPORT LIBRARIES
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# Define paths to the training and test datasets
train_data_path = 'X_train.txt'
train_labels_path = 'y_train.txt'
test_data_path = 'X_test.txt'
test_labels_path = 'y_test.txt'

# Load train and test datasets
X_train = pd.read_csv(train_data_path, sep='\s+', header=None)
y_train = pd.read_csv(train_labels_path, sep='\s+', header=None)
X_test = pd.read_csv(test_data_path, sep='\s+', header=None)
y_test = pd.read_csv(test_labels_path, sep='\s+', header=None)

# Combine training and testing data
X = pd.concat([X_train, X_test], axis=0)
y = pd.concat([y_train, y_test], axis=0)

# One-hot encode the labels since this is a classification task
encoder = OneHotEncoder(sparse_output=False)  # Updated argument name
y = encoder.fit_transform(y)

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Create networks with dropout and without dropout
def create_activity_dropout_and_nodropout_networks(n_inputs: int, n_outputs: int, dropout_rate: float = 0.3, learning_rate: float = 0.001):
    """Creates one neural network with dropout applied after each layer, and
    one neural network without dropout.
    """
    # Network with Dropout
    dropout_input = keras.Input(shape=(n_inputs,))
    dropout_hidden = keras.layers.Dense(128, activation='relu')(dropout_input)
    dropout_hidden = keras.layers.Dropout(dropout_rate)(dropout_hidden)
    dropout_hidden = keras.layers.Dense(64, activation='relu')(dropout_hidden)
    dropout_hidden = keras.layers.Dropout(dropout_rate)(dropout_hidden)
    dropout_hidden = keras.layers.Dense(32, activation='relu')(dropout_hidden)
    dropout_hidden = keras.layers.Dropout(dropout_rate)(dropout_hidden)
    dropout_output = keras.layers.Dense(n_outputs, activation='softmax')(dropout_hidden)
    dropout_model = keras.Model(dropout_input, dropout_output)

    # Network without Dropout
    nodropout_input = keras.Input(shape=(n_inputs,))
    nodropout_hidden = keras.layers.Dense(128, activation='relu')(nodropout_input)
    nodropout_hidden = keras.layers.Dense(64, activation='relu')(nodropout_hidden)
    nodropout_hidden = keras.layers.Dense(32, activation='relu')(nodropout_hidden)
    nodropout_output = keras.layers.Dense(n_outputs, activation='softmax')(nodropout_hidden)
    nodropout_model = keras.Model(nodropout_input, nodropout_output)

    # Compile both models with specified learning rate
    dropout_optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    nodropout_optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    loss = 'categorical_crossentropy'

    dropout_model.compile(optimizer=dropout_optimizer, loss=loss, metrics=['accuracy'])
    nodropout_model.compile(optimizer=nodropout_optimizer, loss=loss, metrics=['accuracy'])

    return dropout_model, nodropout_model

# Hyperparameter Tuning
def hyperparameter_tuning(n_inputs: int, n_outputs: int, X_train, y_train, X_val, y_val):
    """Test different variations of hyperparameters for the dropout and no-dropout networks."""

    # Define hyperparameter sets
    dropout_rates = [0.2, 0.3, 0.4]
    learning_rates = [0.001, 0.002, 0.005]

    best_dropout_model = None
    best_nodropout_model = None
    best_dropout_val_acc = 0
    best_nodropout_val_acc = 0
    best_params = None

    # Loop through the hyperparameter sets
    for dropout_rate in dropout_rates:
        for learning_rate in learning_rates:
            print(f"Testing configuration: Dropout Rate - {dropout_rate}, Learning Rate - {learning_rate}")

            # Create the dropout and no-dropout models for the current configuration
            dropout_model, nodropout_model = create_activity_dropout_and_nodropout_networks(n_inputs, n_outputs, dropout_rate, learning_rate)

            # Train both models
            dropout_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), verbose=1)
            nodropout_model.fit(X_train, y_train, epochs=10, validation_data=(X_val, y_val), verbose=1)

            # Evaluate the models on the validation data
            dropout_val_loss, dropout_val_acc = dropout_model.evaluate(X_val, y_val, verbose=0)
            nodropout_val_loss, nodropout_val_acc = nodropout_model.evaluate(X_val, y_val, verbose=0)

            print(f"Dropout Model Validation Accuracy: {dropout_val_acc}")
            print(f"No Dropout Model Validation Accuracy: {nodropout_val_acc}")

            # Update the best model if this configuration is better
            if dropout_val_acc > best_dropout_val_acc:
                best_dropout_model = dropout_model
                best_dropout_val_acc = dropout_val_acc
                best_params = {'dropout_rate': dropout_rate, 'learning_rate': learning_rate, 'type': 'dropout'}

            if nodropout_val_acc > best_nodropout_val_acc:
                best_nodropout_model = nodropout_model
                best_nodropout_val_acc = nodropout_val_acc
                best_params = {'dropout_rate': dropout_rate, 'learning_rate': learning_rate, 'type': 'nodropout'}

    # Print the best configuration and its validation accuracy
    print(f"Best Model Configuration: {best_params}")
    print(f"Best Dropout Model Validation Accuracy: {best_dropout_val_acc}")
    print(f"Best No Dropout Model Validation Accuracy: {best_nodropout_val_acc}")

    return best_dropout_model, best_nodropout_model

# Use the number of features from the dataset
n_inputs = X_train.shape[1]
n_outputs = y_train.shape[1]  # Number of activity types (6 in this case)

# Run hyperparameter tuning on the dataset
best_dropout_model, best_nodropout_model = hyperparameter_tuning(n_inputs, n_outputs, X_train, y_train, X_val, y_val)


Testing configuration: Dropout Rate - 0.2, Learning Rate - 0.001
Epoch 1/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 5s 7ms/step - accuracy: 0.6712 - loss: 0.8105 - val_accuracy: 0.9500 - val_loss: 0.1391
Epoch 2/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 2s 6ms/step - accuracy: 0.9308 - loss: 0.1902 - val_accuracy: 0.9675 - val_loss: 0.0895
Epoch 3/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.9480 - loss: 0.1426 - val_accuracy: 0.9650 - val_loss: 0.0949
Epoch 4/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9572 - loss: 0.1209 - val_accuracy: 0.9660 - val_loss: 0.0861
Epoch 5/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9636 - loss: 0.1028 - val_accuracy: 0.9684 - val_loss: 0.0889
Epoch 6/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9628 - loss: 0.0928 - val_accuracy: 0.9714 - val_loss: 0.0840
Epoch 7/10
258/258 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9727 - loss: 0.0756 - val_accuracy: 0.9752 - val_loss: 0.0623
Epoch 8/10
258/258 ━━━━━━━━━━━━━━━━━━━━